In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/diamonds/diamonds.csv')
df.head()


In [ ]:
df.describe()

There are zero values in dimentions. This is not possible. To deal with this we can replace 
the zeros with the median with the cut the are of. 

**#For X.**

In [ ]:
df[df['x'] == 0]

In [ ]:
plt.figure(figsize=(15,5))
sns.boxplot(data=df, x='cut', y='x')

In [ ]:
def for_x(cols):
    x = cols[0]
    cut = cols[1]
    if x == 0:
        if cut == 'Ideal':
            return 5.6
        elif cut == 'Premium':
            return 6
        elif cut == 'Good cut':
            return 5.9
        elif cut == 'Very Good':
            return 5.8
        else:
            return 6.1
    else:
        return x

In [ ]:
df['x'] = df[['x', 'cut']].apply(for_x, axis=1)

**For Y.**

In [ ]:
plt.figure(figsize=(15,5))
sns.boxplot(data=df, x='cut', y='y')

In [ ]:
df[(df['cut']=='Ideal')&(df['y'])]['y'].median()

In [ ]:
def for_y(cols):
    y = cols[0]
    cut = cols[1]
    if y == 0:
        if cut == 'Ideal':
            return df[(df['cut']=='Ideal')&(df['y'])]['y'].median()
        elif cut == 'Premium':
            return df[(df['cut']=='Premium')&(df['y'])]['y'].median()
        elif cut == 'Good cut':
            return df[(df['cut']=='Good cut')&(df['y'])]['y'].median()
        elif cut == 'Very Good':
            return df[(df['cut']=='Very Good')&(df['y'])]['y'].median()
        else:
            return df[(df['cut']=='Fair')&(df['y'])]['y'].median()
    else:
        return y

In [ ]:
df['y'] = df[['y', 'cut']].apply(for_y, axis=1)

In [ ]:
df['y'].describe()

**For Z**

In [ ]:
plt.figure(figsize=(15,5))
sns.boxplot(data=df, x='cut', y='z')

In [ ]:
def for_z(cols):
    z = cols[0]
    cut = cols[1]
    if z == 0:
        if cut == 'Ideal':
            return df[(df['cut']=='Ideal')&(df['z'])]['z'].median()
        elif cut == 'Premium':
            return df[(df['cut']=='Premium')&(df['z'])]['z'].median()
        elif cut == 'Good cut':
            return df[(df['cut']=='Good cut')&(df['z'])]['z'].median()
        elif cut == 'Very Good':
            return df[(df['cut']=='Very Good')&(df['z'])]['z'].median()
        else:
            return df[(df['cut']=='Fair')&(df['z'])]['z'].median()
    else:
        return z

In [ ]:
df['z'] = df[['z', 'cut']].apply(for_z, axis=1)

In [ ]:
df['z'].describe()

Although the number zero values was less it was better to replate the '0' rather than drop them.
Now as the value are replaced we need to apply these new changes to the Depth column as well.

In [ ]:
df.describe()

In [ ]:
df['Depth'] = ((2 * df['z']) / (df['x'] + df['y']))*100
df['Depth'] = df['Depth'].round(2)


In [ ]:
df.drop(['Unnamed: 0', 'depth'], axis=1, inplace=True)


In [ ]:
df.head()

In [ ]:
plt.figure(figsize=(15,5))
sns.scatterplot(data=df, x='carat', y='price', hue='cut')

We can see here there is a linear relationship beteen the the price and the carat of a diamond.
Although most of the data lies between 1 to 3 carat. There is no clear seperation among the cut of diamond.

In [ ]:
plt.figure(figsize=(15,5))
sns.scatterplot(data=df, x='carat', y='price', hue='clarity')

Here, we can see a distinction between the clarity of diamond. Same goes for color of diamond.

In [ ]:
#categorical features will be one hot encoded.

cat = pd.get_dummies(df['cut'], drop_first=True)
df.drop(['cut'], axis=1, inplace=True)
df = pd.concat([df, cat], axis=1)
df.head()

According to the description in the dataset the columns of 'clarity' and 'color' have ranking.


In [ ]:
#df['color'].value_counts().keys().to_list()
#df['clarity'].value_counts().keys().to_list()
#after conveting the color, clarity to list creat a dictionary based on the ranking according to description.

color = {'G': 4, 'E': 2, 'F': 3, 'H': 5, 'D': 1, 'I': 6, 'J': 7}
clarity = {'SI1': 6, 'VS2': 5, 'SI2': 7, 'VS1': 4, 'VVS2': 3, 'VVS1': 2, 'IF': 1, 'I1': 8}
df['Color'] = df['color'].map(color)
df['Clarity'] = df['clarity'].map(clarity)
df = df.drop(['color','clarity'], axis=1)

In [ ]:
df.columns

In [ ]:
data = df[['carat', 'table', 'x', 'y', 'z', 'Depth', 'Color', 'Clarity']]
data.head(1)

In [ ]:
#scaling of the data.
#we'll use min max scalar as it bring data into a 0-1 range.
#we will scale all data except the One hot encoded and the price.


from sklearn.preprocessing import MinMaxScaler

scale = MinMaxScaler() 
scale.fit(data)
scaled = scale.transform(data)
scaled = pd.DataFrame(scaled)
scaled.head()

In [ ]:
scaled.columns = ['carat', 'table', 'x', 'y', 'z', 'Depth', 'Color', 'Clarity']
scaled.head(1)

In [ ]:
diamond = pd.concat([scaled, df[['Good','Ideal','Premium','Very Good']], df['price']], axis=1)
diamond.head(5)

Now we have our data ready!
I'll be using linear regression to predict price.

In [ ]:
from sklearn.model_selection import train_test_split
X = diamond.drop('price', axis=1)
y = diamond['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
print(X_train.shape)
print(y_train.shape)



In [ ]:
from sklearn.ensemble import RandomForestRegressor

reg = RandomForestRegressor(n_estimators=200)

reg.fit(X_train, y_train)

y_pred_rf = reg.predict(X_test)

In [ ]:
from sklearn.linear_model import LinearRegression
mod = LinearRegression()

mod.fit(X_train, y_train)

y_pred = mod.predict(X_test)

print(y_pred)

In [ ]:
from sklearn.metrics import r2_score
print(r2_score(y_test, y_pred_rf))
print(r2_score(y_test, y_pred))

In [ ]:
comp = pd.DataFrame({'Actual': y_test, 'Logistc Regression': y_pred, 'RandomForestr regg': y_pred_rf})
comp

In [ ]:
sns.scatterplot(data=comp, x='Actual', y='Logistc Regression')

In [ ]:
sns.scatterplot(data=comp, x='Actual', y='RandomForestr regg')